In [1]:
import pandas as pd

# Load dataset from CSV
df = pd.read_csv("COCO COLA.csv", parse_dates=["Date"])

In [2]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,0.263021,0.270182,0.263021,0.263021,0.051133,806400
1,1962-01-03,0.259115,0.259115,0.253255,0.257161,0.049994,1574400
2,1962-01-04,0.257813,0.261068,0.257813,0.259115,0.050374,844800
3,1962-01-05,0.259115,0.262370,0.252604,0.253255,0.049234,1420800
4,1962-01-08,0.251302,0.251302,0.245768,0.250651,0.048728,2035200
...,...,...,...,...,...,...,...
15091,2021-12-13,56.980000,57.930000,56.959999,57.759998,57.759998,31362800
15092,2021-12-14,57.400002,58.169998,57.400002,57.799999,57.799999,24806600
15093,2021-12-15,57.930000,58.250000,57.650002,58.060001,58.060001,24923800
15094,2021-12-16,57.980000,58.880001,57.900002,58.650002,58.650002,24696900


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15096 entries, 0 to 15095
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       15096 non-null  datetime64[ns]
 1   Open       15096 non-null  float64       
 2   High       15096 non-null  float64       
 3   Low        15096 non-null  float64       
 4   Close      15096 non-null  float64       
 5   Adj Close  15096 non-null  float64       
 6   Volume     15096 non-null  int64         
dtypes: datetime64[ns](1), float64(5), int64(1)
memory usage: 825.7 KB


In [5]:
from pyspark.sql import SparkSession

# Initialize Spark Session
spark = SparkSession.builder.appName("StockData").getOrCreate()

# Load dataset into Spark DataFrame
df_spark = spark.read.csv("COCO COLA.csv", header=True, inferSchema=True)

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("StockData") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "8g") \
    .config("spark.executor.cores", "4") \
    .config("spark.sql.shuffle.partitions", "100") \
    .getOrCreate()

In [6]:
from pyspark.sql.functions import col

# Drop missing values
df_spark = df_spark.dropna()

# Convert data types
df_spark = df_spark.withColumn("Volume", col("Volume").cast("int"))

# Normalize column names
df_spark = df_spark.toDF(*[col.lower().replace(" ", "_") for col in df_spark.columns])

In [8]:
# Save DataFrame to CSV
df.to_csv("coca_cola_clean.csv", index=False)

print("CSV file saved successfully!")

CSV file saved successfully!


In [9]:
df = pd.read_csv('coca_cola_clean.csv')

In [10]:
df

,Date,Open,High,Low,Close,Adj Close,Volume
0,1962-01-02,0.263021,0.270182,0.263021,0.263021,0.051133,806400
1,1962-01-03,0.259115,0.259115,0.253255,0.257161,0.049994,1574400
2,1962-01-04,0.257813,0.261068,0.257813,0.259115,0.050374,844800
3,1962-01-05,0.259115,0.262370,0.252604,0.253255,0.049234,1420800
4,1962-01-08,0.251302,0.251302,0.245768,0.250651,0.048728,2035200
...,...,...,...,...,...,...,...
15091,2021-12-13,56.980000,57.930000,56.959999,57.759998,57.759998,31362800
15092,2021-12-14,57.400002,58.169998,57.400002,57.799999,57.799999,24806600
15093,2021-12-15,57.930000,58.250000,57.650002,58.060001,58.060001,24923800
15094,2021-12-16,57.980000,58.880001,57.900002,58.650002,58.650002,24696900
